### Imports and Installs

In [ ]:
!wget http://images.cocodataset.org/zips/val2017.zip

In [ ]:
#unzip coco dataset
!unzip val2017.zip > /dev/null

In [ ]:
#unzip json instances
!unzip instances_val2017.json.zip > /dev/null

In [ ]:
#install detectron2
!pip install -U torch==1.5 torchvision==0.6 -f https://download.pytorch.org/whl/cu101/torch_stable.html
!pip install pyyaml==5.1 pycocotools>=2.0.1
import torch, torchvision
print(torch.__version__, torch.cuda.is_available())
!gcc --version
!pip install detectron2==0.1.3 -f https://dl.fbaipublicfiles.com/detectron2/wheels/cu101/torch1.5/index.html

In [ ]:
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import numpy as np
import cv2
import random
from google.colab.patches import cv2_imshow
import os
import json
from PIL import Image

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog,DatasetCatalog
from detectron2.structures import BoxMode


### Part 1 : Use Detectron2 to run inference

In [ ]:
from detectron2.data.datasets import register_coco_instances
register_coco_instances("val2017", {}, "/content/instances_val2017.json", "/content/val2017")

In [ ]:
metadata = MetadataCatalog.get("val2017")
dataset_dicts = DatasetCatalog.get("val2017")

In [ ]:
#check that we uploaded the correct data and everything works just fine :)
for d in random.sample(dataset_dicts, 5):
    img = cv2.imread(d["file_name"])
    visualizer = Visualizer(img[:, :, ::-1], metadata=metadata, scale=0.5)
    out = visualizer.draw_dataset_dict(d)
    cv2_imshow(out.get_image()[:, :, ::-1])

### Part 2 : Crop The Sky

In [ ]:
"""
Returns the minimal and maximal pixel of each class

Paramter outputs: represents the outputs after applying panoptic segmentation
"""

In [ ]:
def get_pixels (outputs, id):

  masks = np.asarray(outputs["panoptic_seg"][0].cpu())
  segmentation = np.where(masks == id)
  x_min = int(np.min(segmentation[1]))
  x_max = int(np.max(segmentation[1]))
  y_min = int(np.min(segmentation[0]))
  y_max = int(np.max(segmentation[0]))

  return x_min, x_max, y_min, y_max

In [ ]:
"""
Returns True if the given image pixels contained in the sky
"""


In [ ]:
def item_completely_in_sky(x_min1, x_max1, y_min1, y_max1, x_min2, x_max2, y_min2, y_max2):
  if x_min2< x_min1 and x_max2>x_max1 and y_min2< y_min1 and y_max2>y_max1 :
    return True

In [ ]:
"""
Returns unique id of the sky for each image

"""


In [ ]:
def find_pixel_id (outputs):
  i=0
  id_sky = -1
  for data in outputs["panoptic_seg"][1]:
    if (data['isthing']==False):
      category_id = data['category_id']
      id =  data['id']
      if MetadataCatalog.get(cfg.DATASETS.TRAIN[0]).stuff_classes[category_id] =='sky':
        id_sky = id
        del outputs["panoptic_seg"][1][i]
    i+=1
  return id_sky

In [ ]:
"""
Update and delete items that are completly inside the sky.
This step is for keeping the annotations -
but remove items that won't appear in the image after we change it.
Therefore, we get new list of categories with items that will appear on the
new- cropped image.

Paramter outputs: represents the outputs after applying panoptic segmentation
Parameters x_min2, x_max2, y_min2, y_max2: pixel boundries of the sky in the
image
"""

In [ ]:
def update_categories(outputs, x_min2, x_max2, y_min2, y_max2):
  i=0
  for data in outputs["panoptic_seg"][1]:
    x_min1, x_max1, y_min1, y_max1 = get_pixels(outputs, data['id'])
    if (item_completely_in_sky(x_min1, x_max1, y_min1, y_max1, x_min2, x_max2, y_min2, y_max2)):
      del outputs["panoptic_seg"][1][i]
    i+=1

In [ ]:
"""
Return the pixel boundries of the sky in the image.
Print the updated image on the screen.
"""

In [ ]:
def crop_image(outputs, id, img, d):
  x_min, x_max, y_min, y_max = get_pixels(outputs, id)

  if d > y_max:
    cropped = Image.fromarray(img[y_max:, :, ::-1], mode='RGB')
    cv2_imshow(np.array(cropped))

  else:
    image = Image.fromarray(img[:, :, ::-1], mode='RGB')
    cv2_imshow(np.array(image))

  return x_min, x_max, y_min, y_max

In [ ]:
"""
Manipulate the data such that :
1. Detect the sky unique id in each image
2. Count how many images with sky we have in our dataset (just some interesting
notation)
3. Calculate the sky pixel boundries in the image
4. Update the width of the image corresponding to the upper bound of the sky
5. Update the categories and delete the ones that won't appear in the cropped
image

IN SUMMARY : we crop the sky, update the image, and keep the image annotations-
only the one that will appear in the image after the manipulation

NOTE : due to runtime limitation, I tested my code on 50 random images from
the dataset
"""

In [ ]:
def crop_sky (dataset_dicts, predictor):
  count_images_sky = 0
  i=1
  for d in random.sample(dataset_dicts, 50):
    img = cv2.imread(d["file_name"])
    panoptic_seg, segments_info = predictor(img)["panoptic_seg"]
    outputs = predictor(img)
    id = find_pixel_id(outputs)
    visualizer = Visualizer(img[:, :, ::-1], metadata=metadata)
    out = visualizer.draw_dataset_dict(d)

    if id != -1:
      count_images_sky +=1
      #print new image and update the width
      print(f'Image With Cropped Sky! image num :{i}')
      i+=1
      x2_min, x2_max, y2_min, y2_max = crop_image(outputs, id, out.get_image(),d["width"] )
      if d["width"]>y2_max:
         d["width"] = y2_max
      #update the categories
      update_categories(outputs , x2_min, x2_max, y2_min, y2_max)

    else: #if we didn't change the image, we still want to print it - full size
      print(f'Image Without Cropped Sky. image num :{i}')
      i+=1
      cv2_imshow(out.get_image()[:, :, ::-1])

  return count_images_sky

In [ ]:
"""
In order to detect the sky , I used panoptic segmentation on the dataset
Below is the code to execute the process

Important Note!
I used panoptic segmentation, yet I manipulated and changed the original image,
therefore I didn't change the appearance of the image
(only while testing and learning the dataset :) )
"""

In [ ]:
#use panoptic segmentation on the dataset
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-PanopticSegmentation/panoptic_fpn_R_101_3x.yaml"))
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-PanopticSegmentation/panoptic_fpn_R_101_3x.yaml")
predictor = DefaultPredictor(cfg)

model_final_cafdb1.pkl: 261MB [00:07, 34.4MB/s]                           


In [ ]:
count = crop_sky(dataset_dicts, predictor)

In [ ]:
print(f'Total Images With(out) Sky:{count}')

Total Images With(out) Sky:17
